# Introduction to Deep Reinforcement Learning *via* the Arcade Learning Environment and OpenAI's Gym

/!\ Please run the cell below /!\
--------------------------------------

In [ ]:
import os
JUPYTER_PATH = os.getcwd()

## Dependencies

This notebook requires :
- python 3.X with numpy, matplotlib
- [tensorflow 1.3](https://www.tensorflow.org/install/)
- [OpenAI's Gym](https://github.com/openai/gym)
- [OpenAI's Baselines](https://github.com/openai/baselines) for their PER Implementation (to run the Rainbow algorithm) + their own dependencies

# Content

* Introduction [$\hookrightarrow$](#Introduction)
* A3C [$\hookrightarrow$](#Asynchronous-Advantage-Actor-Critic-(A3C%29)
* DDPG [$\hookrightarrow$](#Deep-Deterministic-Policy-Gradient-(DDPG%29)
* Rainbow DQN [$\hookrightarrow$](#Rainbow-DQN)
* Conclusion [$\hookrightarrow$](#Conclusion)

## Introduction
### Background
A quick recap on reinforcement learning and the notations we will adopt here : we address the problem of an **agent** learning to act in an **environment** in order to maximize a **reward** signal.


At each time step, the environment provides an observation $s_t \in S$, the agent responds by selecting an action $a_t \in A$ and then the environment provides a reward $r_{t+1}$ and the next state $s_{t+1}$ with a discount factor $\gamma \in [0, 1]$.
We assume that this system is a Markov Decision Process e.g. the state of the system at the time step $t+1$ only depends on the state and the action chose at the time step $t$.


The goal of RL is to find a policy $\pi$ (e.g. a probability distribution over actions for each state) that maximize the expected discounted return $G_t = \sum_{k=0}^{+\infty}{\gamma^k R_{t+k+1}}$ over an episode in the environment.

To do so, we learn an estimate of the **Q-Value function** $Q_\pi(s_t, a_t) = E_\pi[G_t|s_t, a_t]$ which is equal to the expected reward an agent will receive starting from a state $s_t$ and action $a_t$ and then acting under policy $\pi$. Once this Q-Value function is known, an optimal policy is trivial : $\pi(s_t) = max_{a \in A}{Q(s_t, a)}$.

Let's also define the **Value function** $V_\pi(s_t) = E_\pi[G_t|s_t]$ the expected discounted reward obtained following a policy $\pi$ starting from a state $s_t$.


*For more details, see the [wikipedia page on MDPs][wikiMDP].*



### DQN Algorithm
Theorically, some methods exist to compute the Q-Value function, such as tabular Q-Learning. The idea is to start from a random guess of the Q-Value of each pair (state, action) and iteratively apply [Bellman Equation][BellEq] until it converges toward the real value of the function *(see [this post by Arthur Juliani][JulianiPost] to get an example)*.


But this method is only appliable on very few problems with tiny state and action spaces, and usual problems generally have nearly infinite large state space, and even if the idea is not bad, the tabular approach cannot work on them.


Deep Reinforcement Learning is an efficient method to solve such problems. The idea is to approximate the Q-Value function with a deep neural network trained by gradient descent to then derive a well performing policy from it.
It was successfully implemented for the first time in 2013 [[Mnih et al][DQN]] as DQN by combining the use of convolutional nets to efficiently processes the raw frames fed as input to the network and of a replay memory buffer not to learn only on immediate rewards.


The optimization by gradient descent is realized wrt to the loss on a randomly chosen time step picked from the replay memory $$\left(R_t + \gamma_{t+1} max_aQ_\bar{\theta}(s_{t+1}, a) - Q\theta(s_t, a_t)\right)^2$$ with $Q_\bar{\theta}$ the *online network* (the network used to select the action) and $Q_\theta$ the *target network* (a copy of the network periodically updated to stabilize the learning).


DQN was the first concrete example of successful reinforcement learning algorithm, and it opened the way to many researches and improvements since 2015. In this notebook, I will present three modified versions of DQN that has lead to improvements or that extended the application domain :
* A3C : an asynchronous version that allows parallelism
* DDPG : a continuous version
* Rainbow DQN : an improved version of DQN, one of the most efficient algorithm of Deep RL today


[wikiMDP]: https://en.wikipedia.org/wiki/Markov_decision_process
[BellEq]: https://en.wikipedia.org/wiki/Bellman_equation#Example
[JulianiPost]: https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0
[DQN]: https://arxiv.org/pdf/1312.5602.pdf

## Asynchronous Advantage Actor-Critic (A3C)

### Concept
Presented in 2016 [[Mnih et al. again](https://arxiv.org/pdf/1602.01783.pdf)], A3C is quite different from regular DQN : it's an Actor-Critic. First, let's debunk the name of the algorithm to understand it better :
+ **Asynchronous :** Asynchronous algorithm is a great family of algos designed to run on parallel architecture to speed the learning up. The main idea is instead of having only one agent interacting with an environment, an asynchronous algo creates a global network and many workers on parallel threads with each their own environment and their own neural network. Then, during an episode, each agent copies the weights of the global network, interacts with it's own environment and apply a gradient descent on the weights of the global network.

 This has two main advantages :
  - the parallelism of today hardware allows to run tens or even hundreds of workers in the same time on different threads, and so to maximize the work done in a given time
  - each worker being independent of the others, they collect a lot of various experience and assure a better exploration of the state space, removing the necessity to have a replay memory buffer and reducing the bias


+ **Actor-Critic :** Instead of just estimating the Q-Value function and then inducing a policy by acting greedily with respect to the action Q-Values, in Actor-Critic algorithms, the network estimate both the Value $V$ function and a policy $\pi$. The value estimator is called the **critic** and the policy estimator the **actor**.



+ **Advantage :** We define advantage as the difference between the Q-Value and the Value of a given state and action $A(s, a) = Q(s, a) - V(s)$ : it represents how much better an action is than expected. During a gradient descent, the updates usually use discounted reward to tell the quality of a taken action, but one way to be more efficient is instead to use an advantage estimate to get how much better it is than on average.

 We can estimate this advantage quite easily because $V(s_t)$ is the output of our actor-critic network and $Q(s_t, a_t)$ can be estimated by the discounted reward $R_t = \sum_{k=0}^{+\infty}{\gamma^k r_{t+k}}$ : $$A_{est}(s_t, a_t) = R_t - V(s_t)$$
 
![](./Images/A3C.png "Architecture of an A3C Network")
<center>*Image from [Arthur Juliani's blog](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-8-asynchronous-actor-critic-agents-a3c-c88f72a5e9f2)*</center>

Additionnaly, to help the network to understand time dependencies, we add a recurrent layer made of LSTM cells (see [Chris Olah's blog post](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) for more details).

### Implementation
*This code can be found in ./BlogFiles/Code/A3C/*

Let's now see an actual implementation of the A3C algorithm applied to a simple Atari 2600 game : Pong.
First, the general outline of the code architecture :
- **Network.py** : defines a class that builds a neural network with convolution and LSTM layers
- **Agent.py** : defines a worker that contains a local network and a local environment and that can interact within this environment to get experiences and train the global network
- **main.py** : the main program that creates a global network and many workers, and run them on different threads
- **Environment.py** : wrapper for [ALE games](https://github.com/mgbellemare/Arcade-Learning-Environment)
- **settings.py** : contains every important parameter that can be modified in the algorithm
- **RMSPropApplier.py** : an object based on [RMS Prop Optimizer](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer) from Tensorflow but that can be shared between the different threads like in the original A3C paper. The code for this object has been taken on [miyosuda's github](https://github.com/miyosuda/async_deep_reinforce/blob/master/rmsprop_applier.py)
- **Displayer.py**, **Saver.py**, **eval.py** : defines visualisation, saving and testing tools


Now, let's analyze more precisely the main lines of the code.
First of all, I'll explain quickly the Environment wrapper because the interface is pretty much the same for the three kinds of algorithms we'll analyze here.<br/>
So the file defines a class Environment with four main methods :
- reset() : resets the environment to a random initial state
- act(action) : perform `action` in the environment and return a tuple `(new_state, reward, done, infos)` (`info` is empty in the environments we'll use)
- set_render(bool) : set the graphic rendering to True or False
- close() : close the environment and the graphic window

Then, let's build a convolutional neural network architecture made of two convolution layers followed by a fully connected and a LSTM network with finally two fully connected layers to estimate the policy (ACTION_SIZE outputs) and the state value (1 output).

**NB :** every variable with capital letters (like ACTION_SIZE) is defined in **settings.py** and we import it in the beginning of each file of the code, which I didn't report here.

In [ ]:
def reset_state():
    """Return a tensor representing an empty LSTM state for a network with 256 cells"""
    return tf.contrib.rnn.LSTMStateTuple(np.zeros([1, 256]),
                                         np.zeros([1, 256]))

def build_layers(device, thread_index):

    scope_name = "net_" + str(thread_index)
    with tf.device(device), tf.variable_scope(scope_name) as scope:
        state = tf.placeholder("float", [None, 84, 84, 4])

        conv1 = tf.layers.conv2d(state, 64, [8, 8],
                                 strides=4, padding="valid",
                                 activation=tf.nn.relu)
        conv2 = tf.layers.conv2d(conv1, 128, [4, 4],s
                                 strides=2, padding="valid",
                                 activation=tf.nn.relu)

        conv2_flat = tf.contrib.layers.flatten(h_conv2)
        hidden = tf.layers.dense(conv2_flat, 256, activation=tf.nn.relu)
        hidden_reshaped = tf.reshape(h_fc1, [1, -1, 256])

        # Placeholder for LSTM unrolling time step size.
        step_size = tf.placeholder(tf.float32, [1])

        lstm = tf.contrib.rnn.BasicLSTMCell(256, state_is_tuple=True)
        
        initial_lstm_state_c = tf.placeholder(tf.float32, [1, 256])
        initial_lstm_state_h = tf.placeholder(tf.float32, [1, 256])
        initial_lstm_state = tf.contrib.rnn.LSTMStateTuple(initial_lstm_state_c,
                                                           initial_lstm_state_h)

        lstm_outputs, lstm_state = tf.nn.dynamic_rnn(lstm,
                                                     hidden_reshaped,
                                                     initial_state=initial_lstm_state,
                                                     sequence_length=step_size,
                                                     time_major=False,
                                                     scope=scope)

        lstm_outputs = tf.reshape(lstm_outputs, [-1, 256])

        # Policy estimation
        policy = tf.layers.dense(lstm_outputs, ACTION_SIZE, activation=tf.nn.softmax)

        # Value estimation
        v_ = tf.layers.dense(lstm_outputs, 1, activation=None)
        value = tf.reshape(v_, [-1])
        
        return policy, value, initial_lstm_state, lstm_state
    
def get_vars(network_scope):
    return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=network_scope)

We can now build the tensorflow operations to compute the policy and value loss. To encourage exploration, we regularize the policy loss with an entropy term that favor the network to explore as long as it lacks information.



In [ ]:
def build_loss():

    action = tf.placeholder("float", [None, ACTION_SIZE])
    reward = tf.placeholder("float", [None])
    td_error = tf.placeholder("float", [None])

    log_pi = tf.log(tf.clip_by_value(policy, 1e-20, 1.0))  # We clip to avoid log(0) error

    entropy = -tf.reduce_sum(policy * log_pi, reduction_indices=1)

    policy_loss = - tf.reduce_sum(tf.reduce_sum(
        tf.multiply(log_pi, action), reduction_indices=1) *
        td_error + entropy * ENTROPY_REG)

    value_loss = 0.5 * tf.nn.l2_loss(reward - value)

    total_loss = policy_loss + value_loss
    return total_loss

Finally, we define a function that copies the weights from a network in a scope and pastes them to another network.

In [ ]:
def copy_network(dst_network, src_network, name=None):
    src_vars = get_vars(src_network)
    dst_vars = get_vars(dst_network)

    copy_ops = []
    with tf.name_scope(name, "Network", []) as name:
        for(src_var, dst_var) in zip(src_vars, dst_vars):
            copy_ops.append(tf.assign(dst_var, src_var))

        return tf.group(*copy_ops, name=name)

We wrap all these functions in a class `Network` and we now can define agents that will run on different threads.

In [ ]:
class Agent:

    def __init__(self,
                 thread_index,
                 global_network,
                 initial_learning_rate,
                 learning_rate_input,
                 grad_applier):

        self.thread_index = thread_index
        self.learning_rate_input = learning_rate_input

        self.local_network = Network(thread_index)
        self.local_network.build_loss()

        local_var_refs = [v._ref() for v in self.local_network.get_vars()]

        self.gradients = tf.gradients(self.local_network.total_loss,
                                      local_var_refs)

        # grad_applier is the shared RMS Prop defined in a separated file 
        self.apply_gradients = grad_applier.apply_gradients(
            global_network.get_vars(),
            self.gradients)

        self.update_network = self.local_network.copy_network(global_network)

        # We create a new environment from the wrapper class of Environment.py
        self.env = Environment(thread_index == 1)
        self.state = self.env.reset()

        self.initial_learning_rate = initial_learning_rate
        self.episode_reward = 0

Now we must train our agent, so we define a `process` method that will make the agent interacts with its environment for at most UPDATE_FREQ steps, then compute the gradient from this direct minibatch of experience and apply it to the global network. At each call of `process`, if the episode was not finished, the agent resumes the last episode and interacts for UPDATE_FREQ more steps.<br/>
We also progressively anneal the learning rate to be finer on the learning and not to take too important steps during the gradient descent.

In [ ]:
def process(self, sess, total_steps):

    buffer = []
    done = False
    episode_step = 0
    start_lstm_state = self.local_network.lstm_state_out

    # Copy weights from global to local network
    sess.run(self.update_network)

    for i in range(UPDATE_FREQ):

        pi, value = self.local_network.run_policy_and_value(sess, self.state)
        a = np.random.choice(ACTION_SIZE, p=pi)
        s_, r, terminal, _ = self.env.process(a)

        self.episode_reward += r
        buffer.append((self.state, a, r, value))

        episode_step += 1
        self.state = s_

        if terminal:
            done = True
            self.episode_reward = 0
            self.env.reset()
            self.local_network.reset_state()
            break

    batch_s = deque()
    batch_a = deque()
    batch_td = deque()
    batch_R = deque()

    # Bootstrapping the discounted reward
    R = 0.0
    if not done:
        R = self.local_network.run_value(sess, self.state)

    # Prepare the memory buffers and compute the N-Step return
    for i in range(len(buffer) - 1, -1, -1):
        si, ai, ri, Vi = buffer[i]
        R = ri + GAMMA * R
        td = R - Vi
        a = np.zeros([ACTION_SIZE])
        a[ai] = 1

        batch_s.appendleft(si)
        batch_a.appendleft(a)
        batch_td.appendleft(td)
        batch_R.appendleft(R)

    cur_learning_rate = self.initial_learning_rate * (MAX_TIME_STEP - total_steps) / MAX_TIME_STEP

    feed_dict = {self.local_network.state: batch_s,
                 self.local_network.action: batch_a,
                 self.local_network.td_error: batch_td,
                 self.local_network.reward: batch_R,
                 self.local_network.initial_lstm_state: start_lstm_state,
                 self.local_network.step_size: [len(batch_a)],
                 self.learning_rate_input: cur_learning_rate}
    sess.run(self.apply_gradients, feed_dict=feed_dict)
    
    return episode_steps

Finally, in **main.py**, we create the global network and several parallel agents and run them asynchronously. Each agent has its own initial learning rate log-uniformly distributed between generally 1e-4 and 1e-2 which reduce the hyperparameter dependency.

In [ ]:
# Return a log-uniform distribution for the learning rates initializer
def log_uniform(lo, hi, rate):
    log_lo = math.log(lo)
    log_hi = math.log(hi)
    v = log_lo * (1 - rate) + log_hi * rate
    return math.exp(v)

def work(worker_index):

    worker = workers[worker_index]
    while not stop_requested and total_steps <= MAX_TIME_STEP:
        steps = worker.process(sess, total_steps)
        total_steps += steps

    worker.close()

# What to do when `Ctrl+C` is sent
def signal_handler(signal, frame):
    global stop_requested
    stop_requested = True


if __name__ == '__main__':

    tf.reset_default_graph()

    initial_learning_rates = log_uniform(1e-4, 1e-2, 0.4)
    stop_requested = False

    # Creation of the global network and the shared RMS Prop Optimizer
    global_network = Network(0)

    learning_rate_input = tf.placeholder("float")

    grad_applier = RMSPropApplier(learning_rate=learning_rate_input,
                                  decay=RMSPROP_DECAY,
                                  momentum=0.0,
                                  epsilon=RMSPROP_EPSILON,
                                  clip_norm=MAX_GRADIENT_NORM,
                                  device=device)

    # Create and initialize the workers
    workers = []
    for i in range(NB_THREADS):
        worker = Agent(i + 1, global_network,
                       initial_learning_rates,
                       learning_rate_input,
                       grad_applier)

        workers.append(worker)

    # prepare session
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    # Create one thread per worker
    train_threads = []
    for i in range(NB_THREADS):
        train_threads.append(threading.Thread(target=work, args=(i,)))

    # Intercept CTRL+C signal
    signal.signal(signal.SIGINT, signal_handler)

    # Start the workers' thread
    for t in train_threads:
        t.start()

    # Wait for interruption
    print('Press Ctrl+C to stop')
    signal.pause()

    for t in train_threads:
        t.join()


### Results
Because of its parallelism, A3C is a great algorithm to solve Atari Games in the Arcade Learning Environment. The most simple game is probably Pong but even with that game it is still quite long to run the algorithm until the solving. For this game, I ran it during one night with 12 parallel threads.<br/>
The left figure shows the evolution of the episode reward of an agent during the training : in the beginning, the agent has a score around -20 which represents 21/1 scores (the winner in Pong is the first one to reach 21 points) whereas in the end it is able to destroy its opponent and win 21 to 0.<br/>
*The interesting curve is the red one, the others are the residues of other learning curves*


![](./Images/A3C_results.png) | ![](./Images/Pong.gif)
:-:                            | :-:
Raw and mean episode rewards   | The agent in action

### Notes
We can visualize on the gif that, even if the learned policy is quite efficient to beat its opponent (it found the weak points : the bouncy shots), the pad is shaky just after it hit the ball. It is because the IA actions won't influence the game at all while the ball is not coming back to the right side, so the algorithm couldn't learn any behavior to have during these phases.<br/>
One way to avoid this shaky behavior would be the implementation of FiGAR [[Sharma et al.](https://arxiv.org/pdf/1702.06054.pdf)] : during the training, the agent learns the actions to perform but also how many times to repeat these actions. So in this situation, it could learn to wait without moving during nearly 1 second just after having hit the ball

## Deep Deterministic Policy Gradient (DDPG)

### Concept
Deterministic Policy Gradient method has been introduced in 2014 [[Silver et al](http://proceedings.mlr.press/v32/silver14.pdf)] and extended as Deep DPG in 2016 [[Lillicrap et al.](https://arxiv.org/pdf/1509.02971.pdf)] to deal with continous control problems. Unlike Atari environments and gaming in general, domains such as robotic need to act continuously on motors for instance, and discretization of the action space is not possible due to the curse of dimensionality.

The idea behind DDPG is quite intuitive : traditionnaly, the method to predict an action over a discrete space is to apply a softmax function over the output of the neural network to get a distribution of probability on the action space.
Here, to build an agent that directly predicts the action to take, we remove the softmax activation function on the output layer and instead apply a sigmoid (or a tanh) that scales the value into [0, 1] (or [-1, 1]). A linear transformation can then rescale this output to the desired interval.

In this architecture, the **critic** is trained as usual with the Bellman Equation, whereas the **actor** is updated by applying the policy gradient presented in the original DPG paper. As in DQN, a target network is also used to evaluate the Q-Value of chosen actions (see [this article section 3](https://medium.com/@awjuliani/simple-reinforcement-learning-with-tensorflow-part-4-deep-q-networks-and-beyond-8438a3e2b8df) by Arthur Juliani for more details on the use of target networks). However, to update the target network, we continually and slowly affect the main network's weights to the target network : $\theta_{target} \leftarrow \tau \theta_{main} + (1 - \tau) \theta_{target}$.

To force exploration, instead of using the usual $\epsilon$-greedy policy, we add a time-correlated noise to action scaled by a decreasing factor : $noise_t = \theta * (\mu - noise_{t-1}) + \sigma * normal(0, 1)$ with $normal$ the normal distribution : $$a_t += noise\_scale_t * noise_t$$

### Implementation
*This code can be found in ./BlogFiles/Code/DDPG/*

The general outline of the code architecture is quite similar to the A3C code, the main difference is that there is now an actor network and a critic network.
Also, we need an experience memory buffer as in DQN which is implemented in **ExperienceBuffer.py**.

First, let's see how to implement the actor network. <br/>
We define a function to generate a neural network with 3 hidden fully connected layer of each 8 neurons which takes in parameters a placeholder for state inputs and two booleans `trainable` (the main network will be, the target one won't) and `reuse` to give the possibility to use the same network with a different input placeholder.<br/>
This network outputs ACTION_SIZE values in [0, 1] (because of the sigmoid function) that we scale between LOW_BOUND and HIGH_BOUND line 17 with a linear transformation.

In [ ]:
# Actor definition :
def generate_actor_network(states, trainable, reuse):
    hidden = tf.layers.dense(states, 8,
                             trainable=trainable, reuse=reuse,
                             activation=tf.nn.relu, name='dense')
    hidden_2 = tf.layers.dense(hidden, 8,
                               trainable=trainable, reuse=reuse,
                               activation=tf.nn.relu, name='dense_1')
    hidden_3 = tf.layers.dense(hidden_2, 8,
                               trainable=trainable, reuse=reuse,
                               activation=tf.nn.relu, name='dense_2')
    actions_unscaled = tf.layers.dense(hidden_3, ACTION_SIZE
                                       trainable=trainable, reuse=reuse,
                                       name='dense_3')
    # bound the actions to the valid range
    valid_range = HIGH_BOUND - LOW_BOUND
    actions = LOW_BOUND + tf.nn.sigmoid(actions_unscaled) * valid_range
    return actions

Unlike in the discrete case, we can't predict the Q-Value of every possible action so our Q Network will also take an action as input to compute $Q(s, a)$. The rest is very similar to the actor definition.

In [ ]:
# Critic definition :
def generate_critic_network(states, actions, trainable, reuse):
    state_action = tf.concat([states, actions], axis=1)
    hidden = tf.layers.dense(state_action, 8,
                             trainable=trainable, reuse=reuse,
                             activation=tf.nn.relu, name='dense')
    hidden_2 = tf.layers.dense(hidden, 8,
                               trainable=trainable, reuse=reuse,
                               activation=tf.nn.relu, name='dense_1')
    hidden_3 = tf.layers.dense(hidden_2, 8,
                               trainable=trainable, reuse=reuse,
                               activation=tf.nn.relu, name='dense_2')
    q_values = tf.layers.dense(hidden_3, 1,
                               trainable=trainable, reuse=reuse,
                               name='dense_3')
    return q_values

Then we define our different placeholders and generate our actor networks :

In [ ]:
# Experience placeholders
state_ph = tf.placeholder(dtype=tf.float32, shape=[None, STATE_SIZE])
action_ph = tf.placeholder(dtype=tf.float32, shape=[None, ACTION_SIZE])
reward_ph = tf.placeholder(dtype=tf.float32, shape=[None])
next_state_ph = tf.placeholder(dtype=tf.float32, shape=[None, STATE_SIZE])
is_not_terminal_ph = tf.placeholder(dtype=tf.float32, shape=[None])

# Main actor network
with tf.variable_scope('actor'):
    actions = generate_actor_network(state_ph, trainable=True, reuse=False)

# Target actor network
with tf.variable_scope('slow_target_actor', reuse=False):
    # tf.stop_gradient is an insurance that the network won't be affected by the gradient descent
    target_next_actions = tf.stop_gradient(
        generate_actor_network(next_state_ph, trainable=False, reuse=False))

The generation of the critic networks is trickier : for the main network, we call the `generate` function twice, once to compute the Q-Value of the actions that we actually took $Q(s, a)$, and once to compute the Q-Value of the actions that our actor network would have chosen $Q(s, Actor(s))$ (used to train the actor network).<br/>
For the temporal difference, like in SARSA, the target network predicts the Q-Value of the next state we were in and the action that the actor would have taken in that state $Q_{target}(s', a')$.

In [ ]:
# Main critic network
with tf.variable_scope('critic') as scope:
    
    # Critic applied to state_ph and action_ph (to train critic as usual)
    q_values_of_given_actions = generate_critic_network(
        state_ph, action_ph, trainable=True, reuse=False)
    
    # Critic applied to state_ph and the current policy's outputted
    # actions for state_ph (to train actor)
    q_values_of_suggested_actions = generate_critic_network(
        state_ph, actions, trainable=True, reuse=True)

# Target critic network
with tf.variable_scope('slow_target_critic', reuse=False):
    # Target critic applied to target actor's outputted
    # actions for next_state_ph (to train critic)
    q_values_next = tf.stop_gradient(
        generate_critic_network(next_state_ph,
                                target_next_actions,
                                trainable=False, reuse=False))

We then compute that temporal difference $$TD = r_t + \gamma Q_{target}(s', a') - Q(s, a)$$

In [ ]:
# One step TD targets y_i for (s,a) from experience replay
# = r_i + GAMMA * Q_target(s',Actor(s')) if s' is not terminal
# = r_i if s' terminal
targets = tf.expand_dims(reward_ph, 1) + \
          tf.expand_dims(is_not_terminal_ph, 1) * GAMMA * q_values_next

td_errors = targets - q_values_of_given_actions

Last, we compute the loss, apply the gradient descent on our networks and define the operations to update the target networks.

In [ ]:
# Isolate vars for each network
actor_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='actor')
target_actor_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_actor')
critic_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='critic')
target_critic_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_critic')


# Critic loss and optimization
critic_loss = tf.reduce_mean(tf.square(td_errors))

critic_trainer = tf.train.AdamOptimizer(CRITIC_LEARNING_RATE)
critic_train_op = critic_trainer.minimize(critic_loss)

# Actor loss and optimization
actor_loss = -1 * tf.reduce_mean(q_values_of_suggested_actions)

actor_trainer = tf.train.AdamOptimizer(ACTOR_LEARNING_RATE)
actor_train_op = actor_trainer.minimize(actor_loss, var_list=self.actor_vars)

# Update values for slowly-changing targets towards current actor and critic
tau = UPDATE_TARGET_RATE
update_target_ops = []
for i, target_actor_var in enumerate(target_actor_vars):
    update_target_actor_op = target_actor_var.assign(tau * actor_vars[i] + (1 - tau) * target_actor_var)
    update_target_ops.append(update_target_actor_op)

for i, target_critic_var in enumerate(target_critic_vars):
    update_target_critic_op = target_critic_var.assign(tau * critic_vars[i] + (1 - tau) * target_critic_var)
    update_target_ops.append(update_target_critic_op)

self.update_targets_op = tf.group(*update_target_ops, name='update_targets')

The experience buffer is just a list to memorize experiences and sample them (we use the data structure *deque* which acts as a list with the extra ability to limit its size not to store too many experiences).

In [ ]:
import numpy as np
import random
from collections import deque

BUFFER_SIZE = 1000000
BATCH_SIZE = 64

class ExperienceBuffer:

    def __init__(self):
        self.buffer = deque(maxlen=BUFFER_SIZE)

    def __len__(self):
        return len(self.buffer)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self):
        return random.sample(self.buffer, BATCH_SIZE)


Then we define the agent with its network and its environment (like in A3C), and a run method :

In [ ]:
def run(self):

    for ep in range(1, TRAINING_STEPS+1):

        episode_reward = 0
        episode_step = 0
        done = False

        # Initialize exploration noise process
        noise_process = np.zeros(ACTION_SIZE)
        noise_scale = (NOISE_SCALE_INIT * NOISE_DECAY**ep) * (HIGH_BOUND - LOW_BOUND)

        # Initial state
        s = self.env.reset()

        while episode_step < MAX_EPISODE_STEPS and not done:

            # choose action based on deterministic policy
            a, = self.sess.run(self.network.actions,
                               feed_dict={self.network.state_ph: s[None]})

            # add temporally-correlated exploration noise to action
            # (using an Ornstein-Uhlenbeck process)
            noise_process = EXPLO_THETA * \
                (EXPLO_MU - noise_process) + \
                EXPLO_SIGMA * np.random.randn(ACTION_SIZE)

            a += noise_scale * noise_process

            s_, r, done, info = self.env.act(a)
            episode_reward += r

            self.buffer.add((s, a, r, s_, 0.0 if done else 1.0))

            # update network weights to fit a minibatch of experience
            if self.total_steps % TRAINING_FREQ == 0 and len(self.buffer) >= BATCH_SIZE:

                minibatch = self.buffer.sample()
                feed_dict = {self.network.state_ph: np.asarray([elem[0] for elem in minibatch]),
                             self.network.action_ph: np.asarray([elem[1] for elem in minibatch]),
                             self.network.reward_ph: np.asarray([elem[2] for elem in minibatch]),
                             self.network.next_state_ph: np.asarray([elem[3] for elem in minibatch]),
                             self.network.is_not_terminal_ph: np.asarray([elem[4] for elem in minibatch])}

                _, _ = self.sess.run([self.network.critic_train_op, self.network.actor_train_op],
                                     feed_dict=feed_dict)

                # update target networks
                _ = self.sess.run(self.network.update_slow_targets_op)

            s = s_
            episode_step += 1


**NB :** In the real code, everything is wrapped up in classes and there are a few more files but it's mostly to set up the interface and get a cleaner code, the core of the algorithm has been presented here.

### Results
A classic environment to test DDPG is Pendulum on [Gym](http://gym.openai.com/envs/Pendulum-v0/).
The goal here is to apply a torque on the pendulum to swing it up so it stays upright.
Execute the cell below to see an exemple of interactions with the environment : we first apply a random torque every time step, and then just apply a counterclockwise torque.

In [ ]:
import gym
env = gym.make("Pendulum-v0")
env.reset()
for i in range(200):
    env.step(env.action_space.sample() if i < 100 else [2])
    env.render()
env.render(close=1)

In that environment, the reward is given at each time step according to the torque applied to the pendulum, so depending on the starting point it may take a few steps to reach the top, that's why the maximum score reachable is in average around -100.

We can easily test our program on that simple environment (it usually takes a few hundred episodes to stabilize around -100)

In [ ]:
import importlib
import os
os.chdir(JUPYTER_PATH + "/Code/DDPG/")
import tensorflow as tf
import Agent
importlib.reload(Agent)

tf.reset_default_graph()
with tf.Session() as sess:

    agent = Agent.Agent(sess)
    try:
        agent.run()
    except KeyboardInterrupt:
        pass
    agent.play(1)
    agent.close()

We can see that this algorithm converges pretty quickly in that simple environment and we can visualize the final policy which is quite effective, even if on the end it sometimes fails to keep the pendulum exactly on the vertical. This could be because the tiny negative reward it gets on the end by keeping a small effort on the pendulum is "hidden" by the large rewards from the swing at the beginning.


![](./Images/DDPG_results.png) | ![](./Images/Pendulum.gif)
:-:                            | :-:
Raw and mean episode rewards   | The agent in action

Another interesting continuous environment is the [Bipedal Walker](http://gym.openai.com/envs/BipedalWalker-v2/) from gym where the goal is to apply torques on each part of the legs of a simple walker to make it go as far as possible. 
From Gym documentation :
> Reward is given for moving forward, total 300+ points up to the far end. If the robot falls, it gets -100. Applying motor torque costs a small amount of points, more optimal agent will get better score. State consists of hull angle speed, angular velocity, horizontal speed, vertical speed, position of joints and joints angular speed, legs contact with ground, and 10 lidar rangefinder measurements. There's no coordinates in the state vector.

In [ ]:
import gym
env = gym.make("BipedalWalker-v2")
env.reset()
for i in range(200):
    env.step(env.action_space.sample())
    env.render()
env.render(close=1)

This environment is more challenging because of the size of the observation space : at each time step, the agent receives 24 different values (instead of 3 for the Pendulum).

We can run the same algorithm to solve this environment, except it will take more time to achieve satisfying results. I ran it for about 6 hours 



Results after 4h                  | Results after 6h
:-:                               | :-:
![](./Images/BipedalWalker_1.gif) | ![](./Images/BipedalWalker_2.gif)

### Notes
* A possible amelioration is to upgrade the experience buffer to a prioritized one [[Schaul et al.](https://arxiv.org/pdf/1511.05952.pdf)] (see [Rainbow DQN section](#Rainbow-DQN) for more details on that).

## Rainbow DQN

### Concept
Since the apparition of DQN in 2013, many improvements have been developped to enhance the basic algorithm such as Double DQN or C51, but no one had tried to combine all these ameliorations until October 2017 with the publication of the Rainbow algorithm [[Hessel et al.](https://arxiv.org/pdf/1710.02298.pdf)].

Five main improvements were used in that algorithm :
- Double DQN [[Van Hasselt et al.](https://arxiv.org/pdf/1509.06461.pdf)]
- Dueling DQN [[Wang et al.](https://arxiv.org/pdf/1511.06581.pdf)]
- Prioritized Experience Replay [[Schaul et al.](https://arxiv.org/pdf/1511.05952.pdf)]
- Distributional DQN [[Bellemare et al.](https://arxiv.org/abs/1707.06887)]
- Noisy DQN [[Fortunato et al.](https://arxiv.org/pdf/1706.10295.pdf)]

![](./Images/Rainbow.jpg)

I won't describe all these enhancements in details since a better description is given on the [related blog post](https://rlsupaero.wordpress.com/2017/11/08/deep-rl-key-papers/) (along other approaches).

### Implementation
*This code can be found in ./BlogFiles/Code/Rainbow/*


The benefit of this algorithm is that it doesn't require a whole new architecture, we can just start from a classical DQN implementation and add features on top of it. That's what I did with the double and the dueling DQN, and with PER but I didn't have time to add Distributional and Noisy DQN yet.

The implementation is quite simple and the general outline of the code is the same as the other codes (A3C and DDPG). As usual, let's analyze the code for the network architecture and the agent.

First, let's build a dueling convolutional network. The network is composed of three convolutional layers followed by two streams of fully connected layers that output the advantage of each action (ACTION_SIZE outputs) and the value of the input state (1 output).

![](./Images/dueling_dqn.png)

In [ ]:
def build_model():

    inputs = tf.placeholder(tf.float32, [None, *STATE_SIZE],
                                 name='Input_state')

    with tf.variable_scope('Convolutional_Layers'):
        conv1 = tf.layers.conv2d(inputs=inputs, filters=32,
                                 kernel_size=[8, 8], strides=[4, 4],
                                 padding='VALID', activation=tf.nn.relu)
        conv2 = tf.layers.conv2d(inputs=conv1, filters=64,
                                 kernel_size=[4, 4], strides=[2, 2],
                                 padding='VALID', activation=tf.nn.relu)
        conv3 = tf.layers.conv2d(inputs=conv2, filters=64,
                                 kernel_size=[3, 3], strides=[1, 1],
                                 padding='VALID', activation=tf.nn.relu)

        # Flatten the output
        hidden = tf.contrib.layers.flatten(conv3)

    return inputs

def dueling(hidden):

    advantage_stream = tf.dense(hidden, 32, activation=tf.nn.relu)
    value_stream = tf.dense(hidden, 32, activation=tf.nn.relu)

    advantage = tf.dense(advantage_stream, ACTION_SIZE, activation=None)
    value = tf.dense(value_stream, 1, activation=None)

    return value, advantage

Then we build the network which combines these two outputs to get the Q-Value and define the operations to train the network.

In [ ]:
with tf.variable_scope("Network"):

    # Define the model as above (wrapped in a class)
    model = NetworkArchitecture(STATE_SIZE, ACTION_SIZE)

    # Convolution network
    inputs = model.build_model()

    # Dueling DQN
    value, advantage = model.dueling()

    # Aggregation of value and advantage to get the Q-value
    adv_mean = tf.reduce_mean(advantage, axis=1, keep_dims=True)
    Qvalues = value + tf.subtract(advantage, adv_mean)

    predict = tf.argmax(Qvalues, 1)

    # Loss
    Qtarget = tf.placeholder(shape=[None], dtype=tf.float32)
    actions = tf.placeholder(shape=[None], dtype=tf.int32)
    actions_onehot = tf.one_hot(actions, ACTION_SIZE, dtype=tf.float32)

    Qaction = tf.reduce_sum(tf.multiply(Qvalues, actions_onehot), axis=1)

    td_error = tf.square(Qtarget - Qaction)
    loss = tf.reduce_mean(td_error)
    
    trainer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
    train = trainer.minimize(loss)


We then define an agent with a main network, a target network and a Prioritized Replay Buffer.
For the PER, I took the [OpenAI implementation](https://github.com/openai/baselines/blob/master/baselines/deepq/replay_buffer.py) from their `baselines` algo bank that implements the right data structure (a SumTree, *cf* the original PER paper for more information) and allows to add experiences and retrieve them efficiently and quickly.

During the initialization of a PrioritizedReplayBuffer, we set the $\alpha$ parameter which determines how much prioritization is used, and each time we sample a minibatch, we provide a $\beta$ parameter called the "importance-sampling parameter" that control the bias introduced by PER. Concretely, we anneal $\beta \rightarrow 1$ to use a strong prioritization at the beginning of the learning (which introduce a strong bias) and progressively uniformize the picking probabilities.

The main methods from the PrioritizedReplayBuffer structure are :
- add(s, a, r, s', terminal)
- sample(batch_size, beta)
- update_priorities(indexes, weights)

Here, we will use the proportional variant of PER that update the weights of experiences based on the TD-error : priority of transition $i$ : $p_i = |\delta_i| + \epsilon$ (to have $p_i > 0$).

Additionnally, we implement N-step learning : instead of using a single reward as a target, we wait for multi-steps to get a better estimation of the reward which slightly modify the loss definition : $$Loss = \left[\sum_{k=0}^{n-1}{\gamma^k r_{t+k}} + \gamma^n \max_a{Q(s_{t+n}, a)} - Q(s_t, a_t)\right]^2$$

In [ ]:
class Agent:

    def __init__(self, sess):
        
        [...]
        self.env = Environment()

        self.mainQNetwork = QNetwork(STATE_SIZE, ACTION_SIZE, 'main')
        self.targetQNetwork = QNetwork(STATE_SIZE, ACTION_SIZE, 'target')
        
        self.buffer = PrioritizedReplayBuffer(BUFFER_SIZE, ALPHA)
        [...]
        
    def run(self):

        self.nb_ep = 1

        while self.nb_ep < TRAINING_STEPS:

            s = self.env.reset()
            episode_reward = 0
            episode_step = 0
            done = False

            memory = deque()

            while episode_step < MAX_EPISODE_STEPS and not done:

                # Epsilon-greedy policy
                if random.random() < self.epsilon:
                    a = random.randint(0, self.action_size - 1)
                else:
                    a = self.sess.run(self.mainQNetwork.predict,
                                      feed_dict={self.mainQNetwork.inputs: [s]})
                    a = a[0]

                s_, r, done, info = self.env.act(a)
                episode_reward += r

                memory.append((s, a, r, s_, done))

                # We have enough experience in the memory to compute the N-Step reward
                if len(memory) > N_STEP_RETURN:
                    s_mem, a_mem, r_mem, ss_mem, done_mem = memory.popleft()
                    discount_R = r_mem
                    for i, (si, ai, ri, s_i, di) in enumerate(memory):
                        discount_R += ri * GAMMA ** (i+1)
                    self.buffer.add(s_mem, a_mem, discount_R, s_, done)

                if episode_step % TRAINING_FREQ == 0:

                    train_batch = self.buffer.sample(BATCH_SIZE, self.beta)

                    # Incr beta
                    if self.beta <= BETA_STOP:
                        self.beta += BETA_INCR

                    feed_dict = {self.mainQNetwork.inputs: train_batch[3]}
                    mainQaction = self.sess.run(self.mainQNetwork.predict,
                                                feed_dict=feed_dict)

                    feed_dict = {self.targetQNetwork.inputs: train_batch[3]}
                    targetQvalues = self.sess.run(self.targetQNetwork.Qvalues,
                                                  feed_dict=feed_dict)

                    done_multiplier = (1 - train_batch[4])
                    doubleQ = targetQvalues[range(BATCH_SIZE), mainQaction]
                    targetQvalues = train_batch[2] + GAMMA * doubleQ * done_multiplier

                    feed_dict = {self.mainQNetwork.inputs: train_batch[0],
                                 self.mainQNetwork.Qtarget: targetQvalues,
                                 self.mainQNetwork.actions: train_batch[1]}
                    td_error, _ = self.sess.run([self.mainQNetwork.train,
                                                  self.mainQNetwork.td_error],
                                                 feed_dict=feed_dict)

                    # Update the priorities of transitions (proportional variant)
                    self.buffer.update_priorities(train_batch[6], td_error + 1e-6)
                    update_target(self.update_target_ops, self.sess)

                s = s_
                episode_step += 1

            self.nb_ep += 1

            # Decay epsilon
            if self.epsilon > EPSILON_STOP:
                self.epsilon -= EPSILON_DECAY

### Results
Another very classic example of environment is [CartPole](http://gym.openai.com/envs/CartPole-v0/) on gym :
> A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. A reward of +1 is provided for every timestep that the pole remains upright. The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.

Let's visualize it by going left and right alternatively :

In [ ]:
import gym
env = gym.make("CartPole-v0")
env.reset()
for i in range(-15, 150):
    env.step((i//30)%2)
    env.render()
env.render(close=1)

Once again, we can try to solve this environment with our program<br/>

(Warning ! The PrioritizedReplayBuffer object form OpenAI's baseline requires the library mpi4py !<br/>
Just run `conda install mpi4py` or `pip install mpi4py` to install this library)

In [ ]:
import importlib

import os
try:os.chdir(JUPYTER_PATH + "/Code/Rainbow/")
except FileNotFoundError:pass

import tensorflow as tf
import Agent
importlib.reload(Agent)

tf.reset_default_graph()
with tf.Session() as sess:

    agent = Agent.Agent(sess)
    sess.run(tf.global_variables_initializer())
    agent.run()
    agent.play(1)
    agent.close()

The learning is quite fast (two minutes at most) due to the simplicity of the environment and the final policy is very effective, reaching the highest possible score almost every time. The few runs where it doesn't reach the highest score are due to the $\epsilon$-greedy exploration policy which is still quite important before 1000 episodes.

![](./Images/Rainbow_results.png) | ![](./Images/CartPole.gif)
:-:                               | :-:
Raw and mean episode rewards      | The agent in action

We can also try to sove another environment [Lunar Lander](http://gym.openai.com/envs/LunarLander-v2/) where the goal is to land a lunar module on a pad by activating its motors.
> Landing pad is always at coordinates (0,0). Coordinates are the first two numbers in state vector. Reward for moving from the top of the screen to landing pad and zero speed is about 100..140 points. If lander moves away from landing pad it loses reward back. Episode finishes if the lander crashes or comes to rest, receiving additional -100 or +100 points. Each leg ground contact is +10. Firing main engine is -0.3 points each frame. Solved is 200 points. Landing outside landing pad is possible. Fuel is infinite, so an agent can learn to fly and then land on its first attempt. Four discrete actions available: do nothing, fire left orientation engine, fire main engine, fire right orientation engine.

In [ ]:
import gym
env = gym.make("LunarLander-v2")
env.reset()
for i in range(120):
    env.step(env.action_space.sample())
    env.render()
env.render(close=1)

As for the Bipedal Walker environment, the computation is way longer than for CartPole (a few hours) but the end results are pretty satisfying

![](./Images/LunarLander_results.png) | ![](./Images/LunarLander.gif)
:-:                                   | :-:
Mean episode rewards          | The agent in action

## Conclusion
We analyzed the implementation of three different kinds of RL algorithm : an asynchronous one, a continuous one and a more basic one. But reinforcement learning is evolving rapidly and these implementation are already starting to be outdated : for instance, the Rainbow algorithm is originally based on Distributional DQN, but the authors of this paper have published an even more efficient algorithm [[Dabney et al.](https://arxiv.org/pdf/1710.10044.pdf)] that could be added to Rainbow.<br/>
Moreover, many smaller methods could be added that slightly improve performances, such as Generalized Advantage Learning or the UNREAL Architecture : these are not keystones but gathered it might make a noticeable difference.<br/>
*see the [blog post](https://rlsupaero.wordpress.com/2017/11/08/deep-rl-key-papers/) for more details about that*